In [1]:
import VariationalBayes as vb
import LogisticGLMM_lib as logit_glmm
import VariationalBayes.SparseObjectives as vb_sparse
import VariationalBayes.ExponentialFamilies as ef

import matplotlib.pyplot as plt
%matplotlib inline

import autograd
import numpy as np
import scipy as sp
from scipy import sparse

import copy
from scipy import optimize

import os
import json

import time

import pickle

from scikits.sparse.cholmod import cholesky


analysis_name = 'simulated_data_small'
#analysis_name = 'criteo_subsampled'

data_dir = os.path.join(os.environ['GIT_REPO_LOC'],
                        'LinearResponseVariationalBayes.py/Models/LogisticGLMM/data')


In [6]:
pickle_output_filename = os.path.join(data_dir, '%s_python_vb_results.pkl' % analysis_name)
pkl_file = open(pickle_output_filename, 'rb')
vb_data = pickle.load(pkl_file)

json_filename = os.path.join(data_dir, '%s_stan_dat.json' % analysis_name)

json_file = open(json_filename, 'r')
json_dat = json.load(json_file)
json_file.close()

stan_dat = json_dat['stan_dat']

K = stan_dat['K'][0]
NObs = stan_dat['N'][0]
NG = stan_dat['NG'][0]
y_g_vec = np.array(stan_dat['y_group'])
y_vec = np.array(stan_dat['y'])
x_mat = np.array(stan_dat['x'])

glmm_par = logit_glmm.get_glmm_parameters(K=K, NG=NG)

# Define a class to contain prior parameters.
prior_par = logit_glmm.get_default_prior_params(K)
prior_par['beta_prior_mean'].set(np.array(stan_dat['beta_prior_mean']))

prior_par['beta_prior_info'].set(np.array(stan_dat['beta_prior_info']))

prior_par['mu_prior_mean'].set(stan_dat['mu_prior_mean'][0])
prior_par['mu_prior_info'].set(stan_dat['mu_prior_info'][0])

prior_par['tau_prior_alpha'].set(stan_dat['tau_prior_alpha'][0])
prior_par['tau_prior_beta'].set(stan_dat['tau_prior_beta'][0])

model = logit_glmm.LogisticGLMM(
    glmm_par, prior_par, x_mat, y_vec, y_g_vec, num_gh_points=vb_data['num_gh_points'])

model.glmm_par.set_free(vb_data['glmm_par_free'])
kl_hess = vb_sparse.unpack_csr_matrix(vb_data['kl_hess_packed'])

In [42]:
sp_kl_hess = \
    model.group_model.get_sparse_kl_vec_hessian(print_every_n=20) + \
    model.global_model.get_sparse_kl_vec_hessian()
kl_hess = np.asarray(sp_kl_hess.todense())

Group 0 of 99.
Group 20 of 99.
Group 40 of 99.
Group 60 of 99.
Group 80 of 99.


In [44]:
kl_hess_inv = np.linalg.inv(kl_hess)

In [40]:
model.global_model.set_global_parameters()
global_indices = model.global_model.global_indices.get_vector()

def get_re_indices(g):
    vec, ind = model.group_model.set_re_parameters([g])
    return ind

get_re_indices([2])

all_re_indices = np.setdiff1d(np.arange(0, model.glmm_par.vector_size()), global_indices)

global_ix = np.ix_(global_indices, global_indices)
re_ix = np.ix_(all_re_indices, all_re_indices)
cross_ix = np.ix_(global_indices, all_re_indices)

# plt.matshow(kl_hess[np.ix_(all_re_indices, all_re_indices)]); plt.colorbar()
# plt.matshow(kl_hess[np.ix_(global_indices, all_re_indices)]); plt.colorbar()
# plt.matshow(kl_hess[np.ix_(global_indices, global_indices)]); plt.colorbar()

In [50]:
global_schur_inv = kl_hess[global_ix] - \
    np.matmul(kl_hess[cross_ix],
              np.linalg.solve(kl_hess[re_ix], kl_hess[cross_ix].T))
global_schur = np.linalg.inv(global_schur_inv)
print(np.linalg.norm(global_schur - kl_hess_inv[global_ix]))

3.70962809744e-10


In [57]:
def get_schur_term(g):
    g_re_ind = get_re_indices([g])
    g_cross_ix = np.ix_(global_indices, g_re_ind)
    g_re_ix = np.ix_(g_re_ind, g_re_ind)
    return np.matmul(kl_hess[g_cross_ix],
                     np.linalg.solve(kl_hess[g_re_ix], kl_hess[g_cross_ix].T))

schur_terms = np.sum(np.array([ get_schur_term(g) for g in range(NG) ]), axis=0)
global_schur_inv = kl_hess[global_ix] - schur_terms
global_schur = np.linalg.inv(global_schur_inv)
print(np.linalg.norm(global_schur - kl_hess_inv[global_ix]))

8.21374294628e-10
